In [3]:
import pandas as pd 
import numpy as np
import neattext.functions as nfx
import seaborn as sn
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

In [4]:
df = pd.read_csv("udemy_courses 2.csv")

In [20]:
df.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,Clean_title
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5,2017-01-18T20:58:58Z,Business Finance,Ultimate Investment Banking Course
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39.0,2017-03-09T16:34:20Z,Business Finance,Complete GST Course Certification Grow Practice
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5,2016-12-19T19:26:30Z,Business Finance,Financial Modeling Business Analysts Consultants
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3.0,2017-05-30T20:07:24Z,Business Finance,Beginner Pro Financial Analysis Excel 2017
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2.0,2016-12-13T14:57:18Z,Business Finance,Maximize Profits Trading Options


In [5]:
df[ 'Clean_title'] = df[ 'course_title'].apply(nfx.remove_stopwords)
df[ 'Clean_title'] = df[ 'Clean_title'].apply(nfx.remove_special_characters)

In [8]:
course_index = pd.Series(df.index, index = df[ 'course_title']).drop_duplicates()

In [11]:
countvect = CountVectorizer()
cv_mat = countvect.fit_transform(df['Clean_title'])

In [13]:
cv_mat.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [17]:
cosine_sim_mat = cosine_similarity(cv_mat)

In [21]:
def recommend_course(title, numrec = 10) :
    course_index = pd. Series (
        df.index, index=df[ 'course_title']).drop_duplicates()
    
    index = course_index[title]
    scores = list(enumerate(cosine_sim_mat [index]))

    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)

    selected_course_index = [i[0] for i in sorted_scores [1:]]

    selected_course_score = [i[1] for i in sorted_scores [1:]]
    
    rec_df = df.iloc[selected_course_index]

    rec_df[ 'Similarity_Score'] = selected_course_score
    
    final_recommended_courses = rec_df[['course_title', 'Similarity_Score', 'url', 'price' , 'num_subscribers']]

    return final_recommended_courses.head(numrec)

In [24]:
ans = recommend_course('Trading Options Basics',4)

/var/folders/41/jsk78lbd6177tft53fbkq64w0000gn/T/ipykernel_49078/3464226883.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec_df[ 'Similarity_Score'] = selected_course_score


In [25]:
ans

,course_title,Similarity_Score,url,price,num_subscribers
95,Options Trading 101: The Basics,0.866025,https://www.udemy.com/options-trading-101-the-...,0,1514
193,Trading Options For Consistent Returns: Option...,0.816497,https://www.udemy.com/trading-options-for-income/,0,4077
861,Basics of Trading,0.816497,https://www.udemy.com/basicsoftrading/,40,99
66,Options Trading Basics (3-Course Bundle),0.774597,https://www.udemy.com/learn-options-trading-co...,180,10100


In [26]:
df.to_csv("Cleaned_data.csv",index=False)